# Imports

In [1]:
import pandas as pd
import model_engine
import boto3
import  numpy as np
from functions_for_onboarding import *

# Load Files

## 's3://power-client-data-staging/CLIENT/PARSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE=inquiry/PULL_NAME=20250201_oefcu_orangecounty_orlando_trustone_vantagewest/'
## 's3://power-client-data-staging/CLIENT/PARSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE=trade/PULL_NAME=20250201_oefcu_orangecounty_orlando_trustone_vantagewest/'


### We can see that both of these lay in the power-client-data-staging bucket since they follow s3://{bucket_name}/{file}

In [2]:
bucket_name = "power-client-data-staging"

In [3]:
files_inquiry_data = list_s3_files(bucket_name = bucket_name, prefix = 'CLIENT/PARSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE=inquiry/PULL_NAME=20250201_oefcu_orangecounty_orlando_trustone_vantagewest/')
files_trade_data = list_s3_files(bucket_name = bucket_name, prefix = 'CLIENT/PARSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE=trade/PULL_NAME=20250201_oefcu_orangecounty_orlando_trustone_vantagewest/')

## Inquiry Data
### Each row: One lender pulled a borrowers credit

In [4]:
test_inquiry_df = load_df_from_list(list = files_inquiry_data, number = 1)

In [5]:
test_inquiry_df.head(5)

,ZEST_KEY,SEG_SEQ,SEG_PARENT,SEG_PARENT_SEQ,SEGMENT_TYPE,DATE_OF_INQUIRY,ABBREVIATION,CUSTOMER_NUMBER,CUSTOMER_NAME,ACCOUNT_TYPE,FILLER1,ARCHIVE_DATA
0,381615_1_192_06,2,FULL-Header,2,FI,12202018,CR,ZB,None,None,None,None
1,381615_1_192_06,3,FULL-Header,3,FI,09072018,CR,ZB,None,None,None,None
2,381615_1_192_06,4,FULL-Header,4,FI,01282020,AR,FP,None,None,None,None
3,381615_1_192_06,5,FULL-Header,5,FI,01202020,AR,FM,None,None,None,None
4,381615_1_192_06,6,FULL-Header,6,FI,01172020,PRM,FM,None,None,None,None


In [6]:
test_inquiry_df.dtypes

ZEST_KEY           object
SEG_SEQ            object
SEG_PARENT         object
SEG_PARENT_SEQ     object
SEGMENT_TYPE       object
DATE_OF_INQUIRY    object
ABBREVIATION       object
CUSTOMER_NUMBER    object
CUSTOMER_NAME      object
ACCOUNT_TYPE       object
FILLER1            object
ARCHIVE_DATA       object
dtype: object

### Abbreviation tells us the context or purpose of the credit inquiry. CR is the only one we care about because that signifies a hard inquiry, the lender (zest_key) applied for credit from the customer who pulled their report as apart of that application

In [7]:
hard_credit_inquiry_df = test_inquiry_df[test_inquiry_df['ABBREVIATION'] == 'CR']

In [8]:
hard_credit_inquiry_df.shape, test_inquiry_df.shape, np.round(len(hard_credit_inquiry_df)/len(test_inquiry_df) * 100,3)

((125357, 12), (500000, 12), 25.071)

### **QUESTION: WHY ARE ALL OF THESE COLUMNS EMPTY? IS IT BECAUSE PII DATA WON'T SHOW**

In [9]:
hard_credit_inquiry_df[["ARCHIVE_DATA", "CUSTOMER_NAME", "SEGMENT_TYPE", "FILLER1",
             "SEG_SEQ", "SEG_PARENT_SEQ", "SEG_PARENT", 'ACCOUNT_TYPE']].value_counts()

Series([], Name: count, dtype: int64)

### Using the inquiry.json to follow, we can drop some columns that we will not need. (They are under exclusion_list)

In [10]:
drop_cols = ["ARCHIVE_DATA", "CUSTOMER_NAME", "SEGMENT_TYPE", "FILLER1",
             "SEG_SEQ", "SEG_PARENT_SEQ", "SEG_PARENT", "ACCOUNT_TYPE"]

hard_credit_inquiry_df = hard_credit_inquiry_df.drop(columns = drop_cols)

### Only 25% of the rows in the inquiry table are from hard credit request pulls

### Clean up the DATE_OF_INQUIRY so that it is in month format

In [11]:
hard_credit_inquiry_df = clean_up_date_column(df = hard_credit_inquiry_df, date_col = 'DATE_OF_INQUIRY')

In [12]:
hard_credit_inquiry_df.head(5)

,ZEST_KEY,DATE_OF_INQUIRY,ABBREVIATION,CUSTOMER_NUMBER
184130,324122_1_192_06,2018-04-01,CR,UZ
147310,98447729_1_084_22,2018-04-01,CR,HZ
97182,118382_1_195_01,2018-04-01,CR,CG
1359,90991399_1_084_03,2018-04-01,CR,ON
365022,102698305_1_193_03,2018-04-01,CR,FA


In [13]:
hard_credit_inquiry_df.dtypes

ZEST_KEY                   object
DATE_OF_INQUIRY    datetime64[ns]
ABBREVIATION               object
CUSTOMER_NUMBER            object
dtype: object

### We can see that the last date available in March 2020. This is the last hard credit request in this data set. Let's create some synthetic date for the application date "DATE_OF_REQUEST"

### Randomly give each Zest_KEY a random value for DATE_OF_REQUEST between April 5th 2020 and June 5th 2020



In [14]:
rng = np.random.default_rng(seed = 42)

unique_keys = pd.Index(hard_credit_inquiry_df['ZEST_KEY'].astype("string").unique())

n_keys = len(unique_keys)
start_ns = pd.Timestamp('2020-04-05').value
end_ns = pd.Timestamp('2020-06-05').value
print(f'start_ns is {start_ns}\n end_ns is {end_ns}')

random_ns = rng.integers(start_ns, end_ns + 1, size=n_keys)

random_dates = pd.to_datetime(random_ns)

date_map = pd.Series(random_dates, index=unique_keys)

hard_credit_inquiry_df['DATE_OF_REQUEST'] =  pd.to_datetime(hard_credit_inquiry_df['ZEST_KEY'].astype('string').map(date_map).dt.date, format = "%m%d%Y", errors = "coerce").dt.normalize()

start_ns is 1586044800000000000
 end_ns is 1591315200000000000


In [15]:
hard_credit_inquiry_df.head(5)

,ZEST_KEY,DATE_OF_INQUIRY,ABBREVIATION,CUSTOMER_NUMBER,DATE_OF_REQUEST
184130,324122_1_192_06,2018-04-01,CR,UZ,2020-05-22
147310,98447729_1_084_22,2018-04-01,CR,HZ,2020-05-01
97182,118382_1_195_01,2018-04-01,CR,CG,2020-05-27
1359,90991399_1_084_03,2018-04-01,CR,ON,2020-05-17
365022,102698305_1_193_03,2018-04-01,CR,FA,2020-04-10


In [16]:
print("pct missing DATE_OF_REQUEST:", hard_credit_inquiry_df["DATE_OF_REQUEST"].isna().mean())
print("pct missing INQUIRY_DATE:", hard_credit_inquiry_df["DATE_OF_INQUIRY"].isna().mean())

pct missing DATE_OF_REQUEST: 0.0
pct missing INQUIRY_DATE: 0.0


In [17]:
hard_credit_inquiry_df.shape

(125357, 5)

### Compute the days since inquiry

In [18]:
hard_credit_inquiry_df['days_from_application'] = (hard_credit_inquiry_df["DATE_OF_REQUEST"] - hard_credit_inquiry_df["DATE_OF_INQUIRY"]).dt.days


In [19]:
hard_credit_inquiry_df['months_from_application'] = hard_credit_inquiry_df["days_from_application"] / 30.4375

### JSON File says the following codes are auto_mortages

### We want to de-dup these rows and only select one of these per time. People often rate shop, when they are looking for these two and this is not neccassarily risky behavior 
"auto_mortgage": [
                        "AZ",
                        "AX",
                        "AU",
                        "AT",
                        "AS",
                        "AR",
                        "AP",
                        "AN",
                        "AL",
                        "AF",
                        "AA",
                        "A",
                        "FA",
                        "FM",
                        "FP",
                        "FS",
                        "FY",
                        "FR",
                        "FC",
                        "FF",
                        "FZ",
                        "FB",
                        "F",
                        "RH",
                        "RE",
                        "RM",
                        "RA",
                        "RZ",
                        "R"
                    ]

## **NEW FOR THIS BRANCH: WE WILL BE MAKING A SET OF AUTO AND MORTAGE CODES AND BE PUTTING PEOPLE INTO ONE OF THE OTHER**


Auto-like: {"AU","AN","A","FA","FF","AR","AT","AS"}

Mortgage-like: {"FM","FR","FS","RE","R","RA","FC"}

In [20]:
AUTO_CODES= {"AU","AN","A","FA","FF","AR","AT","AS"}

MORTGAGE_CODES = {"FM","FR","FS","RE","R","RA","FC"}

hard_credit_inquiry_df["CUSTOMER_NUMBER"] = hard_credit_inquiry_df["CUSTOMER_NUMBER"].astype("string").str.strip()

CONDITIONS = [
    hard_credit_inquiry_df['CUSTOMER_NUMBER'].isin(AUTO_CODES), 
    hard_credit_inquiry_df['CUSTOMER_NUMBER'].isin(MORTGAGE_CODES)
]

choices = ['auto', 'mortgage']

hard_credit_inquiry_df['inquiry_product_type'] = np.select(CONDITIONS, choices, default = 'other')

hard_credit_inquiry_df['is_auto'] = hard_credit_inquiry_df['inquiry_product_type']=='auto'
hard_credit_inquiry_df['is_mortgage'] = hard_credit_inquiry_df['inquiry_product_type']=='mortgage'



hard_credit_inquiry_df['inquiry_product_type'].value_counts(normalize = True)



inquiry_product_type
other       0.602663
auto        0.310729
mortgage    0.086609
Name: proportion, dtype: float64

### Also, we drop utility codes

"additional_drop_rows": [
  {
    "column": "ABBREVIATION",
    "value": ["CR"],
    "op": "not_in",
    "is_optional" : true
  },
  {
    "column": "CUSTOMER_NUMBER",
    "value": ["UW","UT","UF","UG","UC","UZ","UD","UE","UH","UI","U"],
    "op": "in"
  }
]

In [21]:
UTILITY_CODES = {"UW","UT","UF","UG","UC","UZ","UD","UE","UH","UI","U"}
hard_credit_inquiry_df = hard_credit_inquiry_df[~hard_credit_inquiry_df['CUSTOMER_NUMBER'].isin(UTILITY_CODES)]


### Order by ZEST_KEY, whether AUTO or MOrtage and then inquiry date. This will allow us for two rows with the same zest_key and ind_code to grab the previous inquiry dt

In [22]:
## Groups by so ordered by this will allow for shift
df = hard_credit_inquiry_df.sort_values(["ZEST_KEY", "inquiry_product_type", "DATE_OF_INQUIRY"]).copy()

## for each application find the last loan of the same type
### .shift gets the previous row inside the same group
prev = df.groupby(["ZEST_KEY", "inquiry_product_type"])["DATE_OF_INQUIRY"].shift(1)

## find the days since the previous loan
days_since_prev = (df["DATE_OF_INQUIRY"] - prev).dt.days


## mask for when it is an auto or mortage loan

auto_or_mortage = (df["inquiry_product_type"] == "auto") | (df['inquiry_product_type'] =='mortgage')
dup_mask = auto_or_mortage & (~days_since_prev.isna()) & (days_since_prev <= 45)

## De-dup dataset

hard_credit_inquiry_df['days_since_last_inquiry'] = days_since_prev
de_dupped_data = hard_credit_inquiry_df.loc[~dup_mask].copy()

de_dupped_data['months_since_inquiry'] = de_dupped_data['days_since_last_inquiry']/30.4375

### If you examine the row below you can see that 2018-08-21 stays because it was not less than 45 but 2018-08-24	 does not
### Note, when we compute the days_since_prev if it was the first inqury for that person of being either auto_mortage or other, it will be nan
### since we do the last than or equal to it means we don't remove it


# **Question: Seems like when we  de-dup we treat auto and mortage exactly the same, create the same rate-shopping event for both, how come we don't de-dup each separately**


In [23]:
auto_or_morgage_old = (hard_credit_inquiry_df["inquiry_product_type"] == "auto") | (hard_credit_inquiry_df['inquiry_product_type'] =='mortgage')
auto_or_morgage_new = (de_dupped_data["inquiry_product_type"] == "auto") | (de_dupped_data['inquiry_product_type'] =='mortgage')

In [24]:
hard_credit_inquiry_df[(auto_or_morgage_old)].sort_values(by = ["ZEST_KEY", "inquiry_product_type", "DATE_OF_INQUIRY"]).head(15)

,ZEST_KEY,DATE_OF_INQUIRY,ABBREVIATION,CUSTOMER_NUMBER,DATE_OF_REQUEST,days_from_application,months_from_application,inquiry_product_type,is_auto,is_mortgage,days_since_last_inquiry
50833,100000991_1_084_22,2019-03-21,CR,FA,2020-05-16,422,13.864476,auto,True,False,NaN
16998,100001655_1_084_22,2018-08-21,CR,AN,2020-05-15,633,20.796715,auto,True,False,NaN
16996,100001655_1_084_22,2018-08-24,CR,FA,2020-05-15,630,20.698152,auto,True,False,3.0
16994,100001655_1_084_22,2019-01-05,CR,AN,2020-05-15,496,16.295688,auto,True,False,134.0
15228,100008241_2_084_12,2020-02-02,CR,FA,2020-04-13,71,2.332649,auto,True,False,NaN
285173,100020090_1_193_03,2018-12-09,CR,FA,2020-04-13,491,16.131417,auto,True,False,NaN
285171,100020090_1_193_03,2020-02-19,CR,FA,2020-04-13,54,1.774127,auto,True,False,437.0
378703,100023092_1_084_22,2020-02-17,CR,AN,2020-05-12,85,2.792608,auto,True,False,NaN
378704,100023092_1_084_22,2019-02-20,CR,FC,2020-05-12,447,14.685832,mortgage,False,True,NaN
292076,100023092_2_084_22,2019-02-20,CR,FC,2020-04-20,425,13.963039,mortgage,False,True,NaN


In [25]:
hard_credit_inquiry_df[auto_or_morgage_old]['days_since_last_inquiry'].describe()

count    32532.000000
mean        52.874155
std        111.401682
min          0.000000
25%          0.000000
50%          0.000000
75%         38.000000
max        708.000000
Name: days_since_last_inquiry, dtype: float64

In [26]:
de_dupped_data[auto_or_morgage_new]['days_since_last_inquiry'].describe()

count    7817.000000
mean      210.419342
std       137.014160
min        46.000000
25%       103.000000
50%       174.000000
75%       287.000000
max       708.000000
Name: days_since_last_inquiry, dtype: float64

In [27]:
de_dupped_data[auto_or_morgage_new].sort_values(by = 'ZEST_KEY').sort_values(by = ["ZEST_KEY", "inquiry_product_type", "DATE_OF_INQUIRY"]).head(15)

,ZEST_KEY,DATE_OF_INQUIRY,ABBREVIATION,CUSTOMER_NUMBER,DATE_OF_REQUEST,days_from_application,months_from_application,inquiry_product_type,is_auto,is_mortgage,days_since_last_inquiry,months_since_inquiry
50833,100000991_1_084_22,2019-03-21,CR,FA,2020-05-16,422,13.864476,auto,True,False,NaN,NaN
16998,100001655_1_084_22,2018-08-21,CR,AN,2020-05-15,633,20.796715,auto,True,False,NaN,NaN
16994,100001655_1_084_22,2019-01-05,CR,AN,2020-05-15,496,16.295688,auto,True,False,134.0,4.402464
15228,100008241_2_084_12,2020-02-02,CR,FA,2020-04-13,71,2.332649,auto,True,False,NaN,NaN
285173,100020090_1_193_03,2018-12-09,CR,FA,2020-04-13,491,16.131417,auto,True,False,NaN,NaN
285171,100020090_1_193_03,2020-02-19,CR,FA,2020-04-13,54,1.774127,auto,True,False,437.0,14.357290
378703,100023092_1_084_22,2020-02-17,CR,AN,2020-05-12,85,2.792608,auto,True,False,NaN,NaN
378704,100023092_1_084_22,2019-02-20,CR,FC,2020-05-12,447,14.685832,mortgage,False,True,NaN,NaN
292076,100023092_2_084_22,2019-02-20,CR,FC,2020-04-20,425,13.963039,mortgage,False,True,NaN,NaN
105374,100031424_1_084_22,2018-09-06,CR,FA,2020-05-06,608,19.975359,auto,True,False,NaN,NaN


### Create Indicator Variables that Tell How Far Away the Inquires were from the application. Closer means tells us more about the application

In [28]:
de_dupped_data["in_last_1m"]  = de_dupped_data["months_from_application"] <= 1
de_dupped_data["in_last_3m"]  = de_dupped_data["months_from_application"] <= 3
de_dupped_data["in_last_6m"]  = de_dupped_data["months_from_application"] <= 6
de_dupped_data["in_last_9m"]  = de_dupped_data["months_from_application"] <= 9
de_dupped_data["in_last_12m"] = de_dupped_data["months_from_application"] <= 12


In [29]:
de_dupped_data

,ZEST_KEY,DATE_OF_INQUIRY,ABBREVIATION,CUSTOMER_NUMBER,DATE_OF_REQUEST,days_from_application,months_from_application,inquiry_product_type,is_auto,is_mortgage,days_since_last_inquiry,months_since_inquiry,in_last_1m,in_last_3m,in_last_6m,in_last_9m,in_last_12m
147310,98447729_1_084_22,2018-04-01,CR,HZ,2020-05-01,761,25.002053,other,False,False,NaN,NaN,False,False,False,False,False
97182,118382_1_195_01,2018-04-01,CR,CG,2020-05-27,787,25.856263,other,False,False,NaN,NaN,False,False,False,False,False
1359,90991399_1_084_03,2018-04-01,CR,ON,2020-05-17,777,25.527721,other,False,False,NaN,NaN,False,False,False,False,False
365022,102698305_1_193_03,2018-04-01,CR,FA,2020-04-10,740,24.312115,auto,True,False,NaN,NaN,False,False,False,False,False
440045,140500_1_195_02,2018-04-01,CR,ON,2020-05-21,781,25.659138,other,False,False,NaN,NaN,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450558,92376528_1_084_03,2020-03-30,CR,ZB,2020-04-12,13,0.427105,other,False,False,412.0,13.535934,True,True,True,True,True
387424,100612115_1_084_22,2020-03-30,CR,ZB,2020-04-24,25,0.821355,other,False,False,12.0,0.394251,True,True,True,True,True
276380,497168_1_192_22,2020-03-30,CR,FA,2020-04-22,23,0.755647,auto,True,False,NaN,NaN,True,True,True,True,True
223617,82070566_1_084_12,2020-03-30,CR,FC,2020-05-28,59,1.938398,mortgage,False,True,NaN,NaN,False,True,True,True,True


### Prepare Data For Modelling

In [30]:
df_for_modeling = build_final_inquiry_features(df = de_dupped_data)

In [31]:
df_for_modeling.columns

Index(['ZEST_KEY', 'all_num_inq', 'all_mean_gap_days', 'all_std_gap_days',
       'all_gap_cv', 'all_days_since_last_inquiry',
       'all_months_since_last_inquiry', 'all_gap_p10', 'all_gap_p25',
       'all_gap_p50', 'all_gap_p75', 'all_gap_p90', 'all_inq_1m', 'all_inq_3m',
       'all_inq_6m', 'all_inq_9m', 'all_inq_12m', 'all_decay_30d',
       'all_decay_90d', 'auto_num_inq', 'auto_mean_gap_days',
       'auto_std_gap_days', 'auto_gap_cv', 'auto_days_since_last_inquiry',
       'auto_months_since_last_inquiry', 'auto_gap_p10', 'auto_gap_p25',
       'auto_gap_p50', 'auto_gap_p75', 'auto_gap_p90', 'auto_inq_1m',
       'auto_inq_3m', 'auto_inq_6m', 'auto_inq_9m', 'auto_inq_12m',
       'auto_decay_30d', 'auto_decay_90d', 'mortgage_num_inq',
       'mortgage_mean_gap_days', 'mortgage_std_gap_days', 'mortgage_gap_cv',
       'mortgage_days_since_last_inquiry',
       'mortgage_months_since_last_inquiry', 'mortgage_gap_p10',
       'mortgage_gap_p25', 'mortgage_gap_p50', 'mortgage_gap

In [97]:
hard_credit_inquiry_df

,ZEST_KEY,DATE_OF_INQUIRY,ABBREVIATION,CUSTOMER_NUMBER,DATE_OF_REQUEST,days_from_application,months_from_application,inquiry_product_type,is_auto,is_mortgage,days_since_last_inquiry
147310,98447729_1_084_22,2018-04-01,CR,HZ,2020-05-01,761,25.002053,other,False,False,NaN
97182,118382_1_195_01,2018-04-01,CR,CG,2020-05-27,787,25.856263,other,False,False,NaN
1359,90991399_1_084_03,2018-04-01,CR,ON,2020-05-17,777,25.527721,other,False,False,NaN
365022,102698305_1_193_03,2018-04-01,CR,FA,2020-04-10,740,24.312115,auto,True,False,NaN
440045,140500_1_195_02,2018-04-01,CR,ON,2020-05-21,781,25.659138,other,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
450558,92376528_1_084_03,2020-03-30,CR,ZB,2020-04-12,13,0.427105,other,False,False,412.0
387424,100612115_1_084_22,2020-03-30,CR,ZB,2020-04-24,25,0.821355,other,False,False,12.0
276380,497168_1_192_22,2020-03-30,CR,FA,2020-04-22,23,0.755647,auto,True,False,NaN
223617,82070566_1_084_12,2020-03-30,CR,FC,2020-05-28,59,1.938398,mortgage,False,True,NaN


### ZEST_KEY is in format: {Application_ID}_{Number_Applicant (Primary/Secondary)}_{Client_ID}_{Product_ID}


### Canonical product table (from GUAN)

| Product name     | Product ID (last two digits) |
|------------------|------------------------------:|
| Personal loan    | 01                            |
| Direct loan      | 12                            |
| Auto loan        | 02                            |
| Indirect loan    | 22                            |
| Credit card      | 03                            |
| Dark scoring     | 97                            |
| Mortgage         | 04                            |
| Analysis         | 98                            |
| Student loan     | 05                            |
| Other            | 99                            |

### Note, this list in the doc does not totally match what we see

In [88]:
hard_credit_inquiry_df["last_two_digits"] = hard_credit_inquiry_df["ZEST_KEY"].astype("string").str[-2:]
unique_vals = sorted(hard_credit_inquiry_df["last_two_digits"].dropna().unique().tolist())

In [89]:
hard_credit_inquiry_df['last_two_digits'].value_counts(normalize = True)

last_two_digits
22    0.479798
12    0.175993
01    0.143414
03    0.106528
06    0.057907
02    0.028798
88    0.007562
Name: proportion, dtype: Float64

In [91]:
hard_credit_inquiry_df.dtypes

ZEST_KEY                   object
SEG_SEQ                    object
SEG_PARENT                 object
SEG_PARENT_SEQ             object
SEGMENT_TYPE               object
DATE_OF_INQUIRY    datetime64[ns]
ABBREVIATION               object
CUSTOMER_NUMBER            object
CUSTOMER_NAME              object
ACCOUNT_TYPE               object
FILLER1                    object
ARCHIVE_DATA               object
DATE_OF_REQUEST            object
last_two_digits    string[python]
dtype: object